# 流畅的 Python

## 第二部分 数据结构

## 第2章 序列构成的数组

Python 用统一的风格处理序列数据。深入了解 Python 中的不同序列类型，不但能让我们避免重新发明轮子，它们的 API 还能帮助我们把自己定义的 API 设计得跟原生的序列类型一样，或者是跟未来可能出现的序列类型保持兼容。

Python handles sequential data in a unified style. An insight into the different sequence types in Python not only prevents us from reinventing the wheel, but their API also helps us design our own API as a native sequence type, or is compatible with the possible sequence types in the future.

### 2.1 内置序列类型概览

容器序列 list, tuple, collections.deque

扁平序列 str, bytes, bytearray, memoryview, array.array
容器序列里存放的是它们所包含的任意类型的对象的引用，而扁平序列里存放的是值而不是引用，扁平序列其实是一段连续的内存空间。

按照能够被修改来分类：
可变序列 list, bytearray, array.array, collections.deque, memoryview
不可变序列 tuple, str,bytes

### 2.2 列表推导和生成器表达式

通常原则：只用列表推导来创建新的列表，并且尽量保持简短。

句法提示：Python 会忽略代码里[],{}和()中的换行，因此如果你的代码里有多行的列表、列表推导、生成器表达式、字典这一类的，可以省略不太好看的续行符\。

对于 Python 3，列表推导不会再有变量泄漏的问题。列表推导、生成器表达式，以及同它们很相似的集合（set）推导和字典（dict）推导，在 Python 3 中都有了自己的局部作用域，就像函数似的。表达式内部的变量和赋值只在局部作用，表达式的上下文里的同名变量还可以被正常引用，局部变量并不会影响到它们。

列表推导可以帮助我们把一个序列或是其他可迭代类型中的元素过滤或是加工，然后再新建一个列表。

In [2]:
symbols = '@#$%^&*'
codes = [ord(symbol) for symbol in symbols]
codes

[64, 35, 36, 37, 94, 38, 42]

#### 列表推导同 filter 和 map 的比较

In [7]:
symbols = '￥¥£₠￡$'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
print(beyond_ascii)
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
print(beyond_ascii)

[65509, 165, 163, 8352, 65505]
[65509, 165, 163, 8352, 65505]


#### 笛卡尔积

R x S 笛卡尔积是一个列表，列表里的元素是由输入的可迭代类型的元素对构成的元组，因此笛卡尔积列表的长度等于输入变量的长度的乘积。

In [1]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
# 先以颜色排列，再以尺码排列
tshirts = [(color, size) for color in colors
                         for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [2]:
# 先以尺码排列，再以颜色排列
tshirts = [(color, size) for size in sizes
                         for color in colors]
tshirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

#### 生成器表达式

In [3]:
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


In [4]:
# 用生成器表达式初始化元组
symbols = '@#$%^&*'
print(tuple(ord(symbol) for symbol in symbols))

(64, 35, 36, 37, 94, 38, 42)


In [6]:
# 用生成器表达式初始化数组
import array
print(array.array('I', (ord(symbol) for symbol in symbols)))

array('I', [64, 35, 36, 37, 94, 38, 42])


### 2.3 元组不仅仅是不可变的列表

#### 元组和记录

如果把元组当作一些字段的集合，那么数量和位置信息就变得非常重要了。如果在任何元组内对元素排序，这些元素所携带的信息就会丢失，因为这些信息是跟它们的位置有关的。

In [8]:
# 把元组用作记录
lax_cordinates = (33.9, -118.3)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]

# 排序，迭代过程中 passport 变量被绑定到每个元组上
for passport in sorted(traveler_ids):
    # %s 格式运算符能被匹配到对应的元组元素上
    print('%s%s' % passport)
    
# 拆包 unpacking
for country, _ in traveler_ids:
    print(country)

BRACE342567
ESPXDA205856
USA31195855
USA
BRA
ESP


#### 元组拆包



* 元组拆包可以应用到任何迭代对象上，唯一的硬性要求：被可迭代对象的元素数必须要跟接受这些元素的元组的空档数一致
* 应用形式：

> * 平行赋值

> * 不使用中间变量交换两个变量的值

> * 用 \* 运算符把一个可迭代对象拆开作为函数的参数

> * 让一个函数可以以元组的形式返回多个值。然后调用函数的代码就能轻松地接受这些返回值

> * 我们不总是对元组里所有的数据都感兴趣，\_ 占位符可以处理这种情况

> * 用 \* 来处理剩下的元素 (\*args)（平行赋值中，\* 前缀只能用在一个变量名前面，这个变量可以出现在赋值表达式的任意位置）

> * \* 可以应用在嵌套结构中

####  嵌套元组拆包

* 只要这个接受元组的嵌套结构符合表达式本身的嵌套结构，Python 就可以做出正确回应。
* 在 Python 3 之前，元组可以作为形参放在函数声明中，例如 def fn(a, (b, c), d):。然而 Python 3 不再支持这种格式。
* namedtuple 可以解决给记录中的字段命名的问题

In [5]:
from collections import namedtuple

City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', '36.933', (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population='36.933', coordinates=(35.689722, 139.691667))

In [6]:
print(tokyo.population)
print(tokyo.coordinates)

36.933
(35.689722, 139.691667)


#### 具名元组

* 创建一个具名元组需要两个参数，一个是类名，另一个是类的每个字段的名字，后者可以是由数个字符串组成的可迭代对象，或者是由空格分隔开的字段名组成的字符串。
* 存放在对应字段的数据要以一串参数的形式传入到构造函数中（注意：元组的构造函数只接受单一的可迭代对象）
* 可以通过字段名或者位置来获得一个字段的信息
* 除了从普通元组继承来的属性外，具名元组还有一些自己的属性：_fields 属性、类方法 _make(iterable) 和实例方法 _asdict()。
> * _fields 属性是一个包含这个类所有字段名称的元组。
> * 用 _make() 通过接受一个可迭代对象来生成这个类的一个实例
> * _asdict() 把具名元组以 collections.OrderdDict 的形式返回，可以用它把元组里的信息友好地呈现出来。

####  作为不可变列表的元组

* 除了跟增减元素相关的方法之外，元组支持列表的其它所有方法。
* 元组没有 __reversed__ 方法，reversed(my_tuple)这个用法在没有 __reversed__ 的情况下也是合法的。
* 列表和元组的方法及属性对比表

###  2.4 切片

为什么切片和区间会忽略最后一个元素？
> * 当只有最后一个位置信息时，我们也可以快速看出切片和区间里有几个元素，如 range(3) 和 mylist[:3] 都返回3个元素。
> * 当起止位置信息都可见时，我们可以快速计算出切片和区间的长度，用后一个数减去第一个下标（stop-start）即可。
> * 这样做也让我们可以利用任意一个下标来把序列分割成不重叠的两部分，只要写成 my_list[:x] 和 my_list[x:] 就可以了。

#### 对对象进行切片

> * 对 seq[start:stop:step] 进行求值的时候，Python 会调用 seq.\_\_getitem\_\_(slice(start, stop, step))
> * 可以给切片命名来处理纯文本文件

#### 多维切片和省略

> * []运算符里还可以使用以逗号分开的多个索引或者是切片，例如二维的 numpy.ndarray 就可以用 a[i, j] 这种形式来获取，抑或是 a[m:n, k:l]。
> * 要正确处理这种运算符的话，对象的特殊方法 \_\_getitem\_\_ 和 \_\_setitem\_\_ 需要以元组的形式来接收 a[i, j] 中的索引。即 Python 会调用 a.\_\_getitem\_\_((i, j))。
> * 切片还可用来就地修改可变序列，也就是说修改的时候不需要重新组建序列。

#### 给切片赋值

> * 如果把切片放在赋值语句的左边，或把它作为 del 操作的对象，我们就可以对序列进行嫁接、切除或就地修改操作。
> * 如果赋值的对象是一个切片，那么赋值语句的右侧必须是个可迭代对象。即便只有单独一个值，也要把它转换成可迭代的序列。

In [5]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
l[2:5] = [20, 30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [7]:
del l[5:7]
l

[0, 1, 20, 30, 5, 8, 9]

In [8]:
l[3::2] = [11, 22]
l

[0, 1, 20, 11, 5, 22, 9]

In [9]:
l[2:5] = 100

TypeError: can only assign an iterable

In [10]:
l[2:5] = [100]
l

[0, 1, 100, 22, 9]

### 2.5 对序列使用 + 和 * 

在拼接的过程中，两个被操作的序列都不会被修改，Python 会新建一个包含同样类型数据的序列来作为拼接的结果。

如果想要把一个序列复制几份然后再拼接起来，更快捷的做法是把这个序列乘以一个整数。

建立由列表组成的列表
> * 有时我们会需要初始化一个嵌套着几个列表的列表，最好的选择是使用列表推导。
> * 含有指向同一对象的引用的列表是毫无用处的。

In [13]:
l = [1, 2, 3]
l * 5

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [14]:
5 * 'abcd'

'abcdabcdabcdabcdabcd'

In [16]:
# 正确
board = [['_'] * 3 for i in range(3)]
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [18]:
board[1][2] = 'X'
board

[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]

In [19]:
# 错误
weird_board = [['_'] * 3] * 3
weird_board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [20]:
weird_board[1][2] = 'X'
weird_board

[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]

### 2.6 序列的增量赋值 

+= 背后的特殊方法是 \_\_iadd\_\_ （用于“就地加法”）。但是如果一个类没有实现这个方法的话，Python 会退一步调用 \_\_add\_\_。

变量名会不会被关联到新的对象，完全取决于这个类型有没有实现 \_\_iadd\_\_ 这个方法。总体来说，可变序列一般都实现了该方法。

*= 对应 \_\_imul\_\_

对不可变序列进行重复拼接操作的话，效率会很低，因为每次都有一个新对象，而解释器需要把原来对象中的元素先复制到新的对象里，然后再追加新的元素。

####  一个关于 += 的谜题

> * 不要把可变对象放在元组里面。
> * 增量赋值不是一个原子操作。在上述例子中它虽然抛出了异常，但还是完成了操作。
> * 查看 Python 的字节码并不难，而且它对我们了解代码背后的运行机制很有帮助。

### 2.7 list.sort 方法和内置函数 sorted 